<a href="https://colab.research.google.com/github/ekaminuma/ROIS-DS-JOINT/blob/master/datasetautoscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###ROIS-DS-JOINT 2019年度成果プログラム
## **DatasetAutoScan（キーワードによるデータセットWeb Scraping）**

2020/1/30
####神沼英里(東京医科歯科大学 医療データ科学推進室)
---
####**概要**

提案手法である医療アクセス制限研究のオープンデータ順位付けは、全工程が手作業のため高コストである。工程の一部でも自動処理が出来れば、運用がし易い。開発した「DatasetAutoScan」は、Google Dataset Searchのキーワード検索結果からURL情報を抜き出しfigshareのオープンデータを取得するまでを自動処理するLinux+Pythonプログラムである。最初のキーワードに疾患名を入力すると、Figshareのオープンデータを収集する。

####**処理手順**

- [1]. Google Dataset Searchのキーワード検索と結果保存（キーワード例：Diabetes）
- [2]. キーワード検索結果からURLのみ抽出
- [3]. URLリストからhttps://figshare.com/のみ抽出
- [4]. figshare.comのサイトからダウンロードURLを取得
- [5]. データファイル(csv,xlsx等)を取得する。

####**試行結果**

疾患キーワード毎のGoogle Dataset Searchヒット数とFigshare URL数の例

  - 「脳卒中　apoplexy」 : GDS 97件 => Figshare URL 8件
  - 「脊髄小脳変性症：spinocerebellar degeneration」 : GDS 3件 => Figshare URL 0件

####**注意**

Figshare以外の使用を考えないならば、GDSを介さずに直接Figshareの検索エンジンを使う事を利用者は考えるかもしれません。しかしFigshareのキーワード検索を試してみた所、GDS程の実用的な検索結果が得られませんでした。 

####**参考情報**

- Google Dataset Search＝データセットを対象とするGoogleの検索サービス(約2500万データセットを提供) 
- Figshare.com＝研究成果を共有するオープンアクセスリポジトリ

*************************************************
----
----

##**[1] Google Dataset Searchのキーワード検索と結果保存**

- 1-1:Gooogle Dataset Searchのキーワード検索結果取得
- 1-2: Colab DiskへHTMLファイルを書き込み
- 1-3: Local PCへGDS検索結果ファイルをダウンロード
- 1-4: HTMLの中身を表示
^




In [1]:
#---[1-1:Gooogle Dataset Searchのキーワード検索結果取得]-----------------------------------------------------
#
# web_cmdの[query=]の後にキーワードを記入して下さい。 　
#
# query=Heart%20disease [心疾患]
# query=diabetes [糖尿病]
# query=apoplexy [脳卒中]
#  :
#
#-------------------------------------------------------------
# <Response [200]>と表示されれば成功
#--------------------------------------------------------------
import requests

web_cmd='https://datasetsearch.research.google.com/search?query=apoplexy'

geturl = requests.get(web_cmd)
print(geturl)


<Response [200]>


In [2]:
#---[1-2: Colab DiskへHTMLファイルを書き込み]------------------------------
#
#------------------------------------
with open('url_test1.htm', 'w') as f0:
  f0.write(geturl.text)

!ls -l   

total 352
drwxr-xr-x 1 root root   4096 May  4 16:26 sample_data
-rw-r--r-- 1 root root 352458 May  7 00:50 url_test1.htm


In [0]:
#---[1-3: Local PCへGDS検索結果ファイルをダウンロード]------------------------------
#
# ColabからLocal PCへurl_test1.htmをダウンロード
#
#------------------------------------
from google.colab import files

files.download('url_test1.htm')

In [3]:
#---[1-4:htmlの中身確認]------------------------------
#　
# 保存したhtmlファイルの中身を確認する。
#------------------------------------------------------
!head url_test1.htm

<!doctype html><html lang="en" dir="ltr"><head><base href="https://datasetsearch.research.google.com/"><meta name="referrer" content="origin"><link rel="canonical" href="https://datasetsearch.research.google.com/search"><meta name="mobile-web-app-capable" content="yes"><meta name="apple-mobile-web-app-capable" content="yes"><meta name="application-name" content="Google Dataset Search"><meta name="apple-mobile-web-app-title" content="Google Dataset Search"><meta name="apple-mobile-web-app-status-bar-style" content="black"><meta name="msapplication-tap-highlight" content="no"><link rel="manifest" crossorigin="use-credentials" href="_/ResearchSciencesearchDesktopUi/manifest.json"><link rel="home" href="/?lfhs=2"><link rel="msapplication-starturl" href="/?lfhs=2"><link rel="icon" href="//ssl.gstatic.com/images/branding/product/ico/googleg_lodp.ico" sizes="32x32"><link rel="apple-touch-icon-precomposed" href="//ssl.gstatic.com/images/branding/product/ico/googleg_lodp.ico" sizes="32x32"><lin

---
---
##**[2] キーワード検索結果からURL抽出**


- 2-1: HTMLファイルからURLを正規表現(https://)で抽出
- 2-2: URLのsortとuniue化処理
- 2-3:URLリストを表示
- 2-4:URLリストをColabディスクに保存
- 2-5:URLリストをLocal PCへダウンロード

In [8]:
#---------[2-1: HTMLファイルからURLを正規表現(https://)で抽出]-----------------------------
# -*- coding: utf-8 -*-
# HTMLファイルの中身から、URLのみ抽出。
#
# HTMLのタグ構造でURLを取得できないので、
# BeautifulSoupを使わず、正規表現でURLを取得する。
#------------------------------------------------------------------------
import re
 
pattern = "https?://[\w/:%#\$&\?\(\)~\.=\+\-]+"
 
url_list = re.findall(pattern, geturl.text)
print(url_list)
print('LIST LENGTH=%d' % len(url_list))

['https://datasetsearch.research.google.com/', 'https://datasetsearch.research.google.com/search', 'https://www.gstatic.com/_/mss/boq-dataset-search/_/ss/k=boq-dataset-search.ResearchSciencesearchDesktopUi.LL28nv8USFM.L.W.O/am=AlE/d=1/ed=1/ct=zgms/rs=ADoVvm_wp-Vs3UPfe69XULB16pzlaABfxQ/m=immersiveview', 'https://www.gstatic.com/_/mss/boq-dataset-search/_/js/k=boq-dataset-search.ResearchSciencesearchDesktopUi.en_US.mwF6bW5w0XY.es5.O/am=AlE/d=1/excm=_b', 'https://www.gstatic.com/images/branding/googlelogo/svg/googlelogo_clr_74x24px.svg', 'https://www.gstatic.com/images/branding/googlelogo/svg/googlelogo_light_clr_74x24px.svg', 'https://www.gstatic.com/images/branding/googlelogo/svg/googlelogo_dark_clr_74x24px.svg', 'https://www.gstatic.com/images/branding/googlelogo/svg/googlelogo_dark_clr_74x24px.svg', 'https://www.gstatic.com/images/branding/googlelogo/svg/googlelogo_light_clr_74x24px.svg', 'https://lh3.googleusercontent.com/ogw/default-user=s24)', 'https://lh3.googleusercontent.com/ogw

In [9]:
#-----[2-2: URLのsortとuniue化処理]--------------------------------
#
# URLのリストをsortして、unique化
#
#-------------------------------------------------

url_list_uniq = list(set(url_list)) 
url_list2 = sorted(url_list_uniq)
print('LIST LENGTH=%d' % len(url_list2))

LIST LENGTH=97


In [10]:
#---[2-3:URLリストを表示]------------------------
#
# URLリストを確認する
#---------------------------------
import pprint

pprint.pprint(url_list2)

['http://digitalcommons.us/dataset/1999',
 'http://digitalcommons.us/dataset/1999#__sid',
 'http://www.omicsdi.org/dataset/pride/PRD000582',
 'http://www.omicsdi.org/dataset/pride/PXD000015',
 'http://www.w3.org/2000/svg',
 'https://accounts.google.com/ServiceLogin?passive=1209600&amp',
 'https://apis.google.com',
 'https://artsandculture.google.com/?hl=en',
 'https://books.google.com/bkshp?hl=en',
 'https://contacts.google.com/?hl=en',
 'https://creativecommons.org/licenses/by/4.0/',
 'https://datamed.org/display-item.php?repository',
 'https://datasetsearch.research.google.com/',
 'https://datasetsearch.research.google.com/search',
 'https://datasetsearch.research.google.com/search?query%3Dapoplexy',
 'https://datasetsearch.research.google.com/search?query%3Dapoplexy&amp',
 'https://developers.google.com/search/docs/data-types/dataset',
 'https://docs.google.com/document/?usp=docs_alc',
 'https://docs.google.com/presentation/?usp=slides_alc',
 'https://docs.google.com/spreadsheets/?u

In [11]:
#---[2-4:URLリストをColabディスクに保存]------------------------
#
# ファイルに出力:url_list2.txt
#
#--------------------------------------------
str_ = '\n'.join(url_list2)
with open("url_list2.txt", 'wt') as f1:
    f1.write(str_)

!ls -l   
!head url_list2.txt

total 360
drwxr-xr-x 1 root root   4096 May  4 16:26 sample_data
-rw-r--r-- 1 root root   6280 May  7 01:50 url_list2.txt
-rw-r--r-- 1 root root 352458 May  7 00:50 url_test1.htm
http://digitalcommons.us/dataset/1999
http://digitalcommons.us/dataset/1999#__sid
http://www.omicsdi.org/dataset/pride/PRD000582
http://www.omicsdi.org/dataset/pride/PXD000015
http://www.w3.org/2000/svg
https://accounts.google.com/ServiceLogin?passive=1209600&amp
https://apis.google.com
https://artsandculture.google.com/?hl=en
https://books.google.com/bkshp?hl=en
https://contacts.google.com/?hl=en


In [0]:
#---[2-5:URLリストをLocal PCへダウンロード]------------------------------
#
# ColabからLocal PCへurl_list2.txtをダウンロード
#
#------------------------------------
from google.colab import files

files.download('url_list2.txt')


---
---
##**[3]. URLのうちhttps://figshare.com/のみ抽出**

- 3-1: https://figshare.com/のURLのみ抽出
- 3-2: #__sidを除外
- 3-3: figshare URLをLocal PCへダウンロード

In [13]:
#--[3-1: https://figshare.com/のURLのみ抽出]-------------------------------
#
# url_list2.txtからfigshare.comのみ抽出
#---------------------------------------------------------

!grep 'figshare.com' url_list2.txt > url_list2_fs.txt
!cat url_list2_fs.txt 


https://figshare.com/articles/Apoplexy_cerebrovascular_disease_and_stroke_Historical_evolution_of_terms_and_definitions/5772570/1
https://figshare.com/articles/Apoplexy_cerebrovascular_disease_and_stroke_Historical_evolution_of_terms_and_definitions/5772570/1#__sid
https://figshare.com/articles/Physiological_Changes_in_Green_Stems_of_i_Vitis_vinifera_i_L_cv_Chardonnay_in_Response_to_Esca_Proper_and_Apoplexy_Revealed_by_Proteomic_and_Transcriptomic_Analyses/2566051
https://figshare.com/articles/Physiological_Changes_in_Green_Stems_of_i_Vitis_vinifera_i_L_cv_Chardonnay_in_Response_to_Esca_Proper_and_Apoplexy_Revealed_by_Proteomic_and_Transcriptomic_Analyses/2566051#__sid
https://figshare.com/articles/_Surgical_approach_and_postoperative_complications_/1557606/1
https://figshare.com/articles/_Surgical_approach_and_postoperative_complications_/1557606/1#__sid
https://plos.figshare.com/articles/_Incidence_of_PA_in_subgroups_/1557603/1
https://plos.figshare.com/articles/_Incidence_of_PA_in_s

In [14]:
#--------[3-2: #__sidを除外]--------------------------------------
#
# URLの最後に[#__sid]があるURLは、重複なので除外する。
#-------------------------------------------------------------------
!grep -v "__sid$" url_list2_fs.txt > url_list2_fscl.txt
!cat url_list2_fscl.txt
!wc -l url_list2_fscl.txt

https://figshare.com/articles/Apoplexy_cerebrovascular_disease_and_stroke_Historical_evolution_of_terms_and_definitions/5772570/1
https://figshare.com/articles/Physiological_Changes_in_Green_Stems_of_i_Vitis_vinifera_i_L_cv_Chardonnay_in_Response_to_Esca_Proper_and_Apoplexy_Revealed_by_Proteomic_and_Transcriptomic_Analyses/2566051
https://figshare.com/articles/_Surgical_approach_and_postoperative_complications_/1557606/1
https://plos.figshare.com/articles/_Incidence_of_PA_in_subgroups_/1557603/1
https://scielo.figshare.com/articles/A_review_on_the_diagnosis_and_treatment_of_patients_with_clinically_nonfunctioning_pituitary_adenoma_by_the_Neuroendocrinology_Department_of_the_Brazilian_Society_of_Endocrinology_and_Metabolism/7514726/1
https://scielo.figshare.com/articles/Bilateral_acute_visual_loss_from_Rathke_s_cleft_cyst_apoplexy_in_a_patient_with_dengue_fever/7101689/1
https://scielo.figshare.com/articles/Pituitary_apoplexy_pathophysiology_diagnosis_and_management/7511510/1
7 url_list

In [0]:
#---[3-3: figshare URLをLocal PCへダウンロード]------------------------------
# ColabからLocal PCへ「url_list2_fscl.txt」をダウンロード
#
#------------------------------------
from google.colab import files

files.download('url_list2_fscl.txt')

---
---
##**[4]　figshareサイトからオープンデータのURL取得**

GDS検索結果から得た「figshare.com」URLをreadlinesでファイルから読込み、データのダウンロードURLを取得する。

- 4-1:ファイルからfigshareのURLを読み込む
- 4-2:正規表現でオープンデータのURLを抽出
- 4-3:取得ファイルの確認
- 4-4:ファイルからbibのURLを読み込む
- 4-5: bibデータリストをLocal PCへダウンロード



In [15]:
#---[4-1:ファイルからfigshareのURLを読み込む]------------------------------
# 
# https://ndownloader.figshare.com/files/1650964
# 
#
#------------------------------------
import os
str.rstrip(os.linesep)

f = open('url_list2_fscl.txt', 'r', encoding='utf-8')
list = f.readlines()

NHIT=len(list)

nn=1
for tmpline1 in list:
  tmp1 = tmpline1.replace('\n','')
  tmpline = tmp1.replace('\r','')
  print("%d[%s]" % (nn,tmpline)) 
  nn=nn+1
f.close()

print('NHIT=%d' % NHIT)

1[https://figshare.com/articles/Apoplexy_cerebrovascular_disease_and_stroke_Historical_evolution_of_terms_and_definitions/5772570/1]
2[https://figshare.com/articles/Physiological_Changes_in_Green_Stems_of_i_Vitis_vinifera_i_L_cv_Chardonnay_in_Response_to_Esca_Proper_and_Apoplexy_Revealed_by_Proteomic_and_Transcriptomic_Analyses/2566051]
3[https://figshare.com/articles/_Surgical_approach_and_postoperative_complications_/1557606/1]
4[https://plos.figshare.com/articles/_Incidence_of_PA_in_subgroups_/1557603/1]
5[https://scielo.figshare.com/articles/A_review_on_the_diagnosis_and_treatment_of_patients_with_clinically_nonfunctioning_pituitary_adenoma_by_the_Neuroendocrinology_Department_of_the_Brazilian_Society_of_Endocrinology_and_Metabolism/7514726/1]
6[https://scielo.figshare.com/articles/Bilateral_acute_visual_loss_from_Rathke_s_cleft_cyst_apoplexy_in_a_patient_with_dengue_fever/7101689/1]
7[https://scielo.figshare.com/articles/Pituitary_apoplexy_pathophysiology_diagnosis_and_management/

In [16]:
#-----[4-2:正規表現で特定URLを抽出（①https://ndownloader.figshare.com/,②citations/bibtex]------------------------------------
#
# 正規表現で取得したいURLは2種類
#
# ① ndwonloader.figshare.comのURL抽出
#   https://ndownloader.figshare.com/articles/1207864/versions/1
#   
# ②bibtexファイルのURL抽出
#
#-----------------------------------------------------------------------

import requests
import re

nn=1
for tmpline1 in list:
  #-----[1] get url with removed \r\n-------------------------
  tmp1 = tmpline1.replace('\n','')
  tmpline = tmp1.replace('\r','')
  print("%d[%s]" % (nn,tmpline)) 

  #-----[2] get web html-------------------------
  web_cmd=tmpline
  geturl = requests.get(web_cmd)

  #-----[3-①] extract url from web html-------------------------
  pattern = "https?://ndownloader.figshare.com/[\w/:%#\$&\?\(\)~\.=\+\-]+"
  url_list = re.findall(pattern, geturl.text)
  print("%d [%d]" % (nn,len(url_list)))
  
  pattern = "https://ndownloader.figshare.com/articles/\d+?/versions/"
  url_data_hit = re.findall(pattern, geturl.text)
  print("%d [%d]" % (nn,len(url_data_hit)))
  


  #-----[3-②] extract bib from web html-------------------------
  pattern_bib = "articles/\d+?/1/citations/bibtex"
  url_bib_tmp = re.findall(pattern_bib, geturl.text)
  for bib1 in url_bib_tmp:
    tmp1=str(bib1)
    tmp2 = tmp1.replace('\']','')
    url_bib_tmp = tmp2.replace('[\'','')
    url_bib='https://figshare.com/%s' % url_bib_tmp
    print("%d [%s]" % (nn,url_bib))

  #-----[4-1-1] save url--------------
  str_ = '\n'.join(url_list)
  oname='data_list_%d.txt' % nn
  with open(oname, 'wt') as f1:
    f1.write(str_)

  #-----[4-1-2] save outputfile--------------
  str_ = '\n'.join(url_data_hit)
  oname='data_hit_%d.txt' % nn
  with open(oname, 'wt') as f1:
    f1.write(str_)

  #-----[4-1-3] save outputfile--------------
  oname='data_bib_%d.txt' % nn
  with open(oname, 'wt') as f1:
    f1.write(url_bib)

  #-----[5] update-------------------------
  nn=nn+1

1[https://figshare.com/articles/Apoplexy_cerebrovascular_disease_and_stroke_Historical_evolution_of_terms_and_definitions/5772570/1]
1 [9]
1 [1]
1 [https://figshare.com/articles/5772570/1/citations/bibtex]
2[https://figshare.com/articles/Physiological_Changes_in_Green_Stems_of_i_Vitis_vinifera_i_L_cv_Chardonnay_in_Response_to_Esca_Proper_and_Apoplexy_Revealed_by_Proteomic_and_Transcriptomic_Analyses/2566051]
2 [7]
2 [1]
2 [https://figshare.com/articles/2566051/1/citations/bibtex]
3[https://figshare.com/articles/_Surgical_approach_and_postoperative_complications_/1557606/1]
3 [5]
3 [1]
3 [https://figshare.com/articles/1557606/1/citations/bibtex]
4[https://plos.figshare.com/articles/_Incidence_of_PA_in_subgroups_/1557603/1]
4 [0]
4 [0]
4 [https://figshare.com/articles/1557603/1/citations/bibtex]
5[https://scielo.figshare.com/articles/A_review_on_the_diagnosis_and_treatment_of_patients_with_clinically_nonfunctioning_pituitary_adenoma_by_the_Neuroendocrinology_Department_of_the_Brazilian_S

In [25]:
#---[4-3:取得ファイルの確認]----------------------
#
# 1)lsでリストを表示
# 2)1番目のファイルだけ中身を表示する
#
#--------------------------------------------------
!ls -l

!cat data_hit_1.txt
!echo " " 
!cat data_bib_1.txt

total 452
-rw-r--r-- 1 root root    373 May  7 01:51 bib_1.txt
-rw-r--r-- 1 root root    711 May  7 01:51 bib_2.txt
-rw-r--r-- 1 root root    471 May  7 01:51 bib_3.txt
-rw-r--r-- 1 root root    429 May  7 01:51 bib_4.txt
-rw-r--r-- 1 root root    819 May  7 01:51 bib_5.txt
-rw-r--r-- 1 root root    474 May  7 01:51 bib_6.txt
-rw-r--r-- 1 root root    331 May  7 01:51 bib_7.txt
-rw-r--r-- 1 root root     56 May  7 01:51 data_bib_1.txt
-rw-r--r-- 1 root root     56 May  7 01:51 data_bib_2.txt
-rw-r--r-- 1 root root     56 May  7 01:51 data_bib_3.txt
-rw-r--r-- 1 root root     56 May  7 01:51 data_bib_4.txt
-rw-r--r-- 1 root root     56 May  7 01:51 data_bib_5.txt
-rw-r--r-- 1 root root     56 May  7 01:51 data_bib_6.txt
-rw-r--r-- 1 root root     56 May  7 01:51 data_bib_7.txt
-rw-r--r-- 1 root root     59 May  7 01:51 data_hit_1.txt
-rw-r--r-- 1 root root     59 May  7 01:51 data_hit_2.txt
-rw-r--r-- 1 root root     59 May  7 01:51 data_hit_3.txt
-rw-r--r-- 1 root root      0 May  7 01

In [18]:
#---[4-4:ファイルからbibのURLを読み込む]------------------------------
# 
#　データがある個数だけ、bibとndownloaderのURLを読み込む
#---------------------------------------------------------------------
import requests
import re

list_bibfile = []
list_data_idx = []

for kk in range(1, NHIT + 1):
    iname = 'data_bib_%d.txt' % kk
    f = open(iname, 'r', encoding='utf-8')
    list = f.readlines()
    oname='bib_%d.txt' % kk

    nn=1
    for tmpline1 in list:
      tmp_bib0 = tmpline1.replace('\n','')
      tmp_bib_cmd = tmp_bib0.replace('\r','')
      list_bibfile.append(tmp_bib_cmd)
      print("%d:%d[%s]" % (kk,nn,tmp_bib_cmd)) 
      #-----------------------------------
      
      data_article_id = re.match('https://figshare.com/articles/(\d+?)/1/citations/bibtex', tmp_bib_cmd)
      list_data_idx.append(data_article_id)
      #-----------------
      geturl = requests.get(tmp_bib_cmd)
      with open(oname, 'wt') as f0:
        f0.write(geturl.text)

      #--------------
      nn=nn+1

f.close()


bibout = '\n'.join(list_bibfile)
with open('list_bibfile.txt', 'wt') as f2:
  f2.write(str(bibout))



1:1[https://figshare.com/articles/5772570/1/citations/bibtex]
2:1[https://figshare.com/articles/2566051/1/citations/bibtex]
3:1[https://figshare.com/articles/1557606/1/citations/bibtex]
4:1[https://figshare.com/articles/1557603/1/citations/bibtex]
5:1[https://figshare.com/articles/7514726/1/citations/bibtex]
6:1[https://figshare.com/articles/7101689/1/citations/bibtex]
7:1[https://figshare.com/articles/7511510/1/citations/bibtex]


In [0]:
#----[4-5: bibデータリストをLocal PCへダウンロード]
#
# ColabからLocal PCへダウンロード
#
#------------------------------------
from google.colab import files

files.download('list_bibfile.txt')

---
##**[5]. オープンデータのデータファイル(csv,xlsx等)を取得する。**

figshareのダウンロードデータを、data_??,zipとしてコラボディスクに保存。
その後でalldata.zipで1つにまとめて、ローカルPCにダウンロードする。

- 5-1:figshareのダウンロード対象リストを確認
- 5-2:ndownloaderのURLが存在するか確認
- 5-3:figshareのサプリメンタルファイルをダウンロードしてzipファイル(data_??.zip)に保存
- 5-4:複数のdata_??.zipを1つのZIPファイル(alldata.zip)に圧縮
- 5-5: data_*.zipを全部まとめたalldata.zipをLocal PCへダウンロード

In [27]:
#----[5-1:figshareのダウンロード対象リストを確認]--------------------------------
#
#--------------------------------------------
print(list_data_idx)

[<_sre.SRE_Match object; span=(0, 56), match='https://figshare.com/articles/5772570/1/citations>, <_sre.SRE_Match object; span=(0, 56), match='https://figshare.com/articles/2566051/1/citations>, <_sre.SRE_Match object; span=(0, 56), match='https://figshare.com/articles/1557606/1/citations>, <_sre.SRE_Match object; span=(0, 56), match='https://figshare.com/articles/1557603/1/citations>, <_sre.SRE_Match object; span=(0, 56), match='https://figshare.com/articles/7514726/1/citations>, <_sre.SRE_Match object; span=(0, 56), match='https://figshare.com/articles/7101689/1/citations>, <_sre.SRE_Match object; span=(0, 56), match='https://figshare.com/articles/7511510/1/citations>]


In [56]:
#----[5-2:ndownloaderのURLが存在するか確認]--------------------------------------------
#
# data_hist_?.txtの中身を確認
#
#-----------------------------------------------------
import zipfile, urllib.request, shutil
import os

NALL=len(list_bibfile)
NHIT=0
print('NALL=%d' % NALL)
for kk in range(NALL):
  iname = 'data_hit_%d.txt' % (kk+1)
  f = open(iname, 'r', encoding='utf-8')
  list = f.readline()

  if len(list) == 0:
     print('%d: URL IS NULL' % (kk+1))
  else:
    print('%d: %s' % (kk+1, list))
    NHIT=kk+1

NALL=7
1: https://ndownloader.figshare.com/articles/5772570/versions/
2: https://ndownloader.figshare.com/articles/2566051/versions/
3: https://ndownloader.figshare.com/articles/1557606/versions/
4: URL IS NULL
5: URL IS NULL
6: URL IS NULL
7: URL IS NULL


In [55]:
#----[5-3:figshareのサプリメンタルファイルをダウンロードしてzipファイル(data_??.zip)に保存]---------------------------
#
#
#------------------------------------------------------------------------------------------
import zipfile, urllib.request, shutil


for kk in range(NHIT):
  iname = 'data_hit_%d.txt' % (kk+1)
  f = open(iname, 'r', encoding='utf-8')
  list = f.readline()
  
  list_tmp =  list + '1'
  print(list_tmp)

  output_name = 'data_%d.zip' % (kk+1)

  with urllib.request.urlopen(list_tmp) as response, open(output_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(output_name) as zf:
        zf.extractall()
  

https://ndownloader.figshare.com/articles/5772570/versions/1
https://ndownloader.figshare.com/articles/2566051/versions/1
https://ndownloader.figshare.com/articles/1557606/versions/1


In [60]:
#----[5-4:複数のdata_??.zipを1つのZIPファイル(alldata.zip)に圧縮]------------------
#
# 　1)更にLocal PCへダウンロードする為に、1つのZIPにまとめる。
#   2)zipファイルのリストを確認
#
#-------------------------------------------------------------

!zip alldata.zip data_*.zip
!ls -l *.zip

  adding: data_1.zip (stored 0%)
  adding: data_2.zip (stored 0%)
  adding: data_3.zip (stored 0%)
-rw-r--r-- 1 root root 102606 May  7 02:48 alldata.zip
-rw-r--r-- 1 root root  21422 May  7 02:40 data_1.zip
-rw-r--r-- 1 root root  70832 May  7 02:40 data_2.zip
-rw-r--r-- 1 root root   9886 May  7 02:40 data_3.zip


In [0]:
#----[5-5: data_*.zipを全部まとめたalldata.zipをLocal PCへダウンロード]--------------
#
# ColabからLocal PCへダウンロード
#
#-------------------------------------------------------------------------------------
from google.colab import files

files.download('alldata.zip')

---
---
---
